In [1]:
import ee
ee.Initialize()

In [2]:
from geetools import tools, ui, collection, utils

In [3]:
import ipygee as ui

## Get a NDVI Image

In [4]:
l8 = collection.Landsat8SR()

In [5]:
p = ee.Geometry.Point([-72,-42])

In [6]:
col = l8.collection.filterBounds(p)

In [7]:
i = ee.Image(col.first())

In [8]:
ndvi = l8.ndvi(i)

## Get point grid

In [9]:
points = ndvi.sample(scale=10000, geometries=True)

# Linear Function

In [10]:
d = tools.image.linearFunction(ndvi, 'ndvi', mean=0.3, range_min=0, range_max=1, min=2, max=5)
result = ndvi.addBands(d)

In [11]:
chart = ui.chart.Image.bandsByRegion(result, points.limit(100), xProperty='ndvi')

# Gauss Function

In [12]:
gauss = tools.image.gaussFunction(ndvi, 'ndvi', 0, 1, mean=0.3, output_min=0, output_max=1, stretch=2)
gauss = ndvi.addBands(gauss)

In [13]:
chart_gauss = ui.chart.Image.bandsByRegion(gauss, points.limit(100), xProperty='ndvi', bands=['gauss'])

In [14]:
chart_gauss.renderWidget(width=800)

HTML(value='<embed src=data:image/svg+xml;charset=utf-8;base64,PD94bWwgdmVyc2lvbj0nMS4wJyBlbmNvZGluZz0ndXRmLTg…

# Normal Distribution

In [15]:
normal = tools.image.normalDistribution(ndvi, 'ndvi', scale=30, maxPixels=1e13)
result_n = ndvi.addBands(normal)

In [16]:
chart_normal = ui.chart.Image.bandsByRegion(result_n, points.limit(100), xProperty='ndvi', bands=['normal_distribution'])

# Charts

In [17]:
charts = chart.cat(chart_gauss, chart_normal)

In [18]:
charts.renderWidget(width=800)

HTML(value='<embed src=data:image/svg+xml;charset=utf-8;base64,PD94bWwgdmVyc2lvbj0nMS4wJyBlbmNvZGluZz0ndXRmLTg…

In [19]:
charts.dataframe

,normal_distribution,gauss,linear_function
-0.264706,0.499408,0.005691,0.193277
-0.088890,0.567368,0.088583,0.444442
-0.078514,0.570398,0.100673,0.459266
-0.003532,0.588462,0.228679,0.566383
-0.000902,0.588968,0.234580,0.570140
-0.000405,0.589063,0.235707,0.570850
0.000149,0.589168,0.236966,0.571641
0.000299,0.589197,0.237309,0.571856
0.000335,0.589204,0.237391,0.571908
0.001249,0.589376,0.239480,0.573213
